In [2]:

import pandas as pd
import numpy as np
from io import StringIO
import pytz
from datetime import datetime, timezone
from zoneinfo import ZoneInfo


Imagina que eres un investigador en el mundo de Pokémon y has recibido una base de datos con información desactualizada y desordenada sobre avistamientos de Pokémon. Tu misión es limpiar y actualizar esta base de datos para que pueda ser utilizada en un estudio sobre la población de Pokémon en la región.

Datos Iniciales

Los datos iniciales contienen las siguientes columnas:

    SightingDate: Fecha del avistamiento.
    TrainerID: Identificación del entrenador que reportó el avistamiento.
    PokemonName: Nombre del Pokémon avistado.
    CP: Puntos de combate del Pokémon reportado.
    HP: Puntos de salud del Pokémon reportado.
    Type: Tipo del Pokémon.
    Weather: Clima durante el avistamiento.

# 1 Carga de Datos

In [6]:
path_datos = '/pokemon.csv'
# TODO: completa el código para cargar los datos
df = pd.read_csv(path_datos, encoding='latin-1')
print(df)


               SightingDate    SightingTimeUTC TrainerID PokémonName       CP  \
0  2023-11-08T14:00:00+0000  2023-11-08T14:00Z     TR123     Pikachu      500   
1  2023-07-12T09:30:00+0100  2023-07-12T08:30Z     TR456       Eevee  MISSING   
2  2023/02/23T22:15:00+0900  2023-02-23T13:15Z     TR789    Magicarp    1,000   
3  2023-04-30T06:45:00-0400  2023-04-30T10:45Z     TR101      Gengar      800   
4  2023-08-15T16:00:00+1000                NaN     TR102   Bulbasaur      750   

   HP          Type        Weather  
0  35      Electric          Clear  
1  55        Normal         Cloudy  
2  10         Water           Rain  
3  45         Ghost  PARTLY_CLOUDY  
4  50  Grass/Poison          Sunny  


# Limpieza de Datos

## 1 Normalizacion de Zonas Horarias

Normaliza la columna `'SightingTimeUTC'` a la zona horaria UTC y convierte `'SightingDate'` al mismo formato de tiempo.


In [9]:
df['SightingTimeUTC'] = pd.to_datetime(df['SightingTimeUTC'],errors="coerce", utc=True)
df['SightingDate'] = pd.to_datetime(df['SightingDate'], errors="coerce", utc=True)
print(df)



               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  
0      500  35      Electric          Clear  
1  MISSING  55        Normal         Cloudy  
2    1,000  10         Water           Rain  
3      800  45         Ghost  PARTLY_CLOUDY  
4      750  50  Grass/Poison          Sunny  


Compara si la fecha de la columna `'SightingDate'` coincide con la fecha en `'SightingTimeUTC'` una vez normalizada.


In [10]:
# Pista: Puedes crear una columna nueva para el resultado de la comparación.
df['DateMatch'] = df['SightingDate'] == df['SightingTimeUTC']
print(df)



               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  DateMatch  
0      500  35      Electric          Clear       True  
1  MISSING  55        Normal         Cloudy       True  
2    1,000  10         Water           Rain       True  
3      800  45         Ghost  PARTLY_CLOUDY       True  
4      750  50  Grass/Poison          Sunny      False  


Ajusta `'SightingTimeUTC'` a la zona horaria local de cada entrenador y crea una columna `'SightingTimeLocal'`.

In [11]:
df['SightingTimeLocal'] = df['SightingTimeUTC'].dt.tz_convert('Mexico/General')
print(df)



               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  DateMatch  \
0      500  35      Electric          Clear       True   
1  MISSING  55        Normal         Cloudy       True   
2    1,000  10         Water           Rain       True   
3      800  45         Ghost  PARTLY_CLOUDY       True   
4      750  50  Grass/Poison          Sunny      False   

          SightingTimeLocal  
0 2023-11-08 08:00:00-06:00  
1 2023-07-12 02:30:00-06:00  
2 2023-02-23 07:15:00-06:00  
3 2023-04-30 04:45:00-06:00  
4                       Na

Calcula el tiempo transcurrido desde el momento del avistamiento hasta `'ahora'` (tu hora local) y crea una columna `'TimeSinceSighting'`.

In [13]:
now = datetime.now(timezone.utc)

df['TimeSinceSighting'] = now - df['SightingTimeLocal']
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  DateMatch  \
0      500  35      Electric          Clear       True   
1  MISSING  55        Normal         Cloudy       True   
2    1,000  10         Water           Rain       True   
3      800  45         Ghost  PARTLY_CLOUDY       True   
4      750  50  Grass/Poison          Sunny      False   

          SightingTimeLocal        TimeSinceSighting  
0 2023-11-08 08:00:00-06:00   7 days 08:31:33.796990  
1 2023-07-12 02:30:00-06:00 126 days 14:01:33.796990  
2 2023-02-2

## 2. Limpeiza de IDs

Llena los valores faltantes en `'TrainerID'` con el ID `'UNKNOWN'`.


In [14]:
df['TrainerID'] = df['TrainerID'].fillna('UNKNOWN')
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  DateMatch  \
0      500  35      Electric          Clear       True   
1  MISSING  55        Normal         Cloudy       True   
2    1,000  10         Water           Rain       True   
3      800  45         Ghost  PARTLY_CLOUDY       True   
4      750  50  Grass/Poison          Sunny      False   

          SightingTimeLocal        TimeSinceSighting  
0 2023-11-08 08:00:00-06:00   7 days 08:31:33.796990  
1 2023-07-12 02:30:00-06:00 126 days 14:01:33.796990  
2 2023-02-2

## 3. Corrección de Nombres de Pokémon

Asegúrate de que los nombres de Pokémon estén capitalizados correctamente.


In [15]:
df['PokémonName'] = df['PokémonName'].str.capitalize()
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  DateMatch  \
0      500  35      Electric          Clear       True   
1  MISSING  55        Normal         Cloudy       True   
2    1,000  10         Water           Rain       True   
3      800  45         Ghost  PARTLY_CLOUDY       True   
4      750  50  Grass/Poison          Sunny      False   

          SightingTimeLocal        TimeSinceSighting  
0 2023-11-08 08:00:00-06:00   7 days 08:31:33.796990  
1 2023-07-12 02:30:00-06:00 126 days 14:01:33.796990  
2 2023-02-2

## 4. Conversión de 'CP' y 'HP' a Numéricos

Convierte `'CP'` y `'HP'` a valores numéricos, manejando los `'MISSING'` y comas como separadores de miles.

In [16]:
df['CP'] = df['CP'].replace(',', '', regex=True)
df['CP'] = df['CP'].replace('MISSING', np.nan)

df['HP'] = df['HP'].replace(',', '', regex=True)
df['HP'] = df['HP'].replace('MISSING', np.nan)

print(df)



               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

     CP  HP          Type        Weather  DateMatch         SightingTimeLocal  \
0   500  35      Electric          Clear       True 2023-11-08 08:00:00-06:00   
1   NaN  55        Normal         Cloudy       True 2023-07-12 02:30:00-06:00   
2  1000  10         Water           Rain       True 2023-02-23 07:15:00-06:00   
3   800  45         Ghost  PARTLY_CLOUDY       True 2023-04-30 04:45:00-06:00   
4   750  50  Grass/Poison          Sunny      False                       NaT   

         TimeSinceSighting  
0   7 day

## 5. Estandarización de 'Type'

Divide la columna `'Type'` en `'PrimaryType'` y `'SecondaryType'` cuando hay dos tipos.


In [17]:
df[['PrimaryType', 'SecondaryType']] = df['Type'].str.split('/', expand=True)
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

     CP  HP          Type        Weather  DateMatch         SightingTimeLocal  \
0   500  35      Electric          Clear       True 2023-11-08 08:00:00-06:00   
1   NaN  55        Normal         Cloudy       True 2023-07-12 02:30:00-06:00   
2  1000  10         Water           Rain       True 2023-02-23 07:15:00-06:00   
3   800  45         Ghost  PARTLY_CLOUDY       True 2023-04-30 04:45:00-06:00   
4   750  50  Grass/Poison          Sunny      False                       NaT   

         TimeSinceSighting PrimaryType

## 6. Corrección del Clima

Estándariza la columna `'Weather'` para que todos los valores sean mayúsculas.


In [18]:
df['Weather'] = df['Weather'].str.upper()
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

     CP  HP          Type        Weather  DateMatch         SightingTimeLocal  \
0   500  35      Electric          CLEAR       True 2023-11-08 08:00:00-06:00   
1   NaN  55        Normal         CLOUDY       True 2023-07-12 02:30:00-06:00   
2  1000  10         Water           RAIN       True 2023-02-23 07:15:00-06:00   
3   800  45         Ghost  PARTLY_CLOUDY       True 2023-04-30 04:45:00-06:00   
4   750  50  Grass/Poison          SUNNY      False                       NaT   

         TimeSinceSighting PrimaryType

# 3 Analisis de Datos

## Agrupaciones

Agrupa el DataFrame por `'Type'` y calcula la suma de `'CP'` para cada grupo.

In [19]:
# Tu codigo aqui
tipo = df.groupby('Type')['CP'].sum().reset_index()
print(tipo)


           Type    CP
0      Electric   500
1         Ghost   800
2  Grass/Poison   750
3        Normal     0
4         Water  1000


Después de la suma, agrega una columna que calcule la media de `'HP'` por cada `'Type'`, pero solo para aquellos Pokémon cuyo `'CP'` sea mayor que el promedio de `'CP'` de todo el DataFrame.

In [21]:
# Convertir la columna 'CP' a tipo numérico
df['CP'] = pd.to_numeric(df['CP'], errors='coerce')

# Calcular la suma de 'CP' por 'Type'
suma_cp_por_tipo = df.groupby('Type')['CP'].sum().reset_index()

# Calcular el promedio de 'CP' en todo el DataFrame
promedio_cp_global = df['CP'].mean()

# Filtrar Pokémon cuyo 'CP' es mayor que el promedio global
pokemon_cp_superior_promedio = df[df['CP'] > promedio_cp_global]

# Calcular la media de 'HP' por 'Type' para Pokémon con 'CP' superior al promedio global
media_hp_por_tipo = pokemon_cp_superior_promedio.groupby('Type')['HP'].mean().reset_index()

# Fusionar los resultados en un solo DataFrame
resultado_final = pd.merge(suma_cp_por_tipo, media_hp_por_tipo, on='Type', suffixes=('_CP', '_HP'))

# Mostrar el resultado final
print(resultado_final)


    Type      CP    HP
0  Ghost   800.0  45.0
1  Water  1000.0  10.0
